# LAB511: Build Agentic Knowledge Bases - Next Level RAG with Azure AI Search

This lab demonstrates Azure AI Search's agentic retrieval capabilities through 8 hands-on notebook exercises. You'll work with Knowledge Bases that connect chat models to multiple data sources and use intelligent agents to decompose queries, retrieve relevant content, and synthesize grounded answers with citations.

<details>
<summary><b>🤖 What is Agentic RAG?</b></summary>

Unlike traditional RAG, agentic retrieval analyzes conversation context to understand user intent, decomposes complex queries into focused subqueries, and executes searches concurrently across multiple sources. It ranks and filters results using semantic understanding, then synthesizes answers with citations back to source documents.

</details>
<details>
<summary><b>🧠 What is a Knowledge Base?</b></summary>

In Azure AI Search, a **knowledge base** is a top-level resource that connects a chat completion model to one or more **knowledge sources** (searchable indexes or remote data sources). It defines which data sources to query, which model to use for reasoning, and how query execution should be optimized. Once created, you can update it at any time.

</details>

## Lab Notebooks

<details><summary>📚 This lab contains 8 notebooks, each focusing on a specific knowledge base configuration or optimization pattern. Extend to learn more!</summary>

- **Part 1: Basic Knowledge Base**: Create a knowledge source, configure a knowledge base with Azure OpenAI, and run queries with citation-backed answers.

- **Part 2: Multiple Knowledge Sources**: Query multiple indexes. Use `always_query_source`, `retrieval_instructions`, and `answer_instructions` for control.

- **Part 3: SharePoint Knowledge Source**: Connect to SharePoint documents with `RemoteSharePointKnowledgeSource` for real-time querying.

- **Part 4: Web Knowledge Source**: Query public web URLs with `WebKnowledgeSource`. Combine internal and external content.

- **Part 5: Blob Knowledge Source**: Upload from Azure Blob Storage. Compare minimal (fast) vs. standard (semantic) indexing.

- **Part 6: Combined Knowledge Sources**: Query search indexes, web URLs, and SharePoint simultaneously.

- **Part 7: Minimal Reasoning**: Use `KnowledgeRetrievalMinimalReasoningEffort` to optimize for speed and cost.

- **Part 8: Medium Reasoning**: Use `KnowledgeRetrievalMediumReasoningEffort` for balanced query decomposition.

</details>

## Pre-configured Environment

This lab uses the following Azure resources, already set up for you:

- **Azure AI Search**
  - `hrdocs` index: HR policies, handbook, company info
  - `healthdocs` index: health benefits, insurance, coverage
  - Semantic ranking enabled, pre-computed embeddings

- **Azure OpenAI**
  - `gpt-4.1`: Chat completion for reasoning and synthesis
  - `text-embedding-3-large`: Embedding model for vector search

- **Azure Storage & Services**
  - Blob Storage, Azure AI Services, SharePoint access

All documents are already indexed and vectorized. You'll focus on the agentic retrieval APIs, not data preparation.

> **📌 Tips** 
>
> - Start with Part 1 to understand core concepts, then continue with Parts 2-6 to explore different knowledge source types, and finally Parts 7-8 to optimize reasoning effort for performance.
>
> - Each notebook is self-contained, you can run them independently or in sequence.
>
> - ‼️ **Read the outputs**. Activity logs show how agentic retrieval works under the hood.
>
>

Let's start with Part 1! 👇

# Part 1: Basic Knowledge Base

In this part, you'll create a knowledge source connected to the `hrdocs` index in Azure AI Search. Then, you'll set up a knowledge base using this source and an Azure OpenAI model. Finally, you'll run queries to see how agentic retrieval produces citation-backed answers.

## Step 1: Load Environment Variables

Load the configuration for your Azure resources. The `.env` file in the workspace root has everything you need: Azure AI Search endpoints, Azure OpenAI credentials, and the knowledge base name.

> **ℹ️ Note** 
>
> - The first time you run the cell below, you'll be prompted to select Kernel, select **Python Environments** and finally choose the **.venv(3.11.9)** environment that is created for you.
>
> - You will also be prompted with "Do you want to allow public and private networks to access this app?" Select **Allow**.

> **⚠️ Troubleshooting**
>
> If code cells get stuck and keep spinning, select **Restart** from the notebook toolbar at the top. If the issue persists after a couple of tries, close VS Code completely and reopen it.

In [5]:
import os

from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "hrdocs-knowledge-base"
knowledge_source_name = "hrdocs-knowledge-source"

# Azure OpenAI configuration (identity-based auth, no API key needed)
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

Environment variables loaded


## Step 2: Create HR Docs Knowledge Source

A **knowledge source** connects your knowledge base to actual data. It points to a specific index and tells Azure AI Search which fields to use.

The code below creates a knowledge source for the `hrdocs` index (documents with HR policies, employee handbook, and company info). The `source_data_fields` specify:
- `blob_path`: Where the original document lives (used for citation links)
- `snippet`: The actual text content

In [6]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndexFieldReference, SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for HR Docs Knowledge",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name="hrdocs",
        source_data_fields=[SearchIndexFieldReference(name="blob_path"), SearchIndexFieldReference(name="snippet")]
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'hrdocs-knowledge-source' created or updated successfully.


## Step 3: Create HR Docs Knowledge Base

A **knowledge base** is the orchestration layer that combines:

1. Your data sources (the knowledge source you just created in Step 2)
2. An AI model (Azure OpenAI) for understanding queries and generating answers
3. Configuration for how to process queries and format responses

The `output_mode=ANSWER_SYNTHESIS` tells the knowledge base to use the AI model to synthesize natural language answers from the retrieved documents, rather than just returning raw search results. The LLM decides what information is relevant and how to present it.

Think of the knowledge base as the intelligent agent that uses your knowledge source (the data) and the AI model (the reasoning engine) to answer questions.

In [7]:
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalOutputMode, KnowledgeSourceReference

# Using identity-based auth: the search service's system-assigned managed identity
# authenticates to Azure OpenAI (no api_key needed)
aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name
)

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name
        )
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

Knowledge base 'hrdocs-knowledge-base' created or updated successfully.


## Step 4: Use agentic retrieval to fetch results

Now you're ready to query the knowledge base. When you send a query, the knowledge base doesn't just do a simple search. Instead, it:

1. Analyzes the entire conversation to infer the user's information need.

1. Decomposes the query into focused subqueries.

1. Runs the subqueries concurrently against your knowledge source.

1. Uses semantic ranker to rerank and filter the results.

1. Synthesizes the top results into a natural-language answer.

The parameters you set below control this behavior:
- `include_references=True`: Return citations showing which documents were used
- `include_reference_source_data=True`: Include the actual text snippets, not just references
- `always_query_source=True`: Always search the index, even if the model thinks it knows the answer
- `include_activity=True`: Show us the detailed activity log of what the agent did

Let's run the code below to ask: "What is the responsibility of the Zava CEO?" and see how the agent processes this query.

In [14]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, SearchIndexKnowledgeSourceParams

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)

req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="What is the responsibility of the Zava CEO?")])
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            include_references=True,
            include_reference_source_data=True,
            always_query_source=True
        )
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
print(f"Retrieved content from '{knowledge_base_name}' successfully.")

Retrieved content from 'hrdocs-knowledge-base' successfully.


## Step 5: Review response, references, and activity log

The knowledge base returns three key pieces of information:

1. **Response**: The synthesized natural language answer to your question
2. **References**: Citations showing which documents and text snippets were used to generate the answer
3. **Activity log**: A detailed trace of the agentic retrieval process - what subqueries were generated, which sources were searched, how results were ranked, etc.

Reviewing these outputs helps you:
- **Trust**: See exactly where the answer came from (transparency)
- **Debug**: Understand why certain sources were or weren't used
- **Optimize**: Learn how to tune your knowledge base configuration

Run the cells below to examine each component.

In [17]:
from IPython.display import display, Markdown

display(Markdown(result.response[0].content[0].text))

The Chief Executive Officer (CEO) at Zava is responsible for providing strategic direction and oversight to ensure the company’s long-term success and profitability. Key responsibilities include developing and implementing strategies and objectives, guiding the executive team, managing day-to-day operations, ensuring compliance with laws and regulations, overseeing marketing and advertising strategies, managing relationships with key stakeholders, representing the company at public events, negotiating contracts, monitoring industry trends, identifying growth opportunities, and maintaining a positive company culture [ref_id:0].

In [18]:
import json

references = json.dumps([ref.as_dict() for ref in result.references], indent=2)
print(references)

[
  {
    "type": "searchIndex",
    "id": "0",
    "activity_source": 1,
    "source_data": {
      "uid": "810256edf976_aHR0cHM6Ly9tYWdvdHRlaWFkbHNnZW4yLmJsb2IuY29yZS53aW5kb3dzLm5ldC9sYWJkb2NzL3JvbGVfbGlicmFyeS5wZGY1_pages_0",
      "blob_path": "/hrdocs/role_library.pdf",
      "snippet": "Roles Descriptions at \n\nZava \n\n  \n\n\n\nThis document contains information generated using a language model (Azure OpenAI). The \n\ninformation contained in this document is only for demonstration purposes and does not \n\nreflect the opinions or beliefs of Microsoft. Microsoft makes no representations or \n\nwarranties of any kind, express or implied, about the completeness, accuracy, reliability, \n\nsuitability or availability with respect to the information contained in this document. \n\nAll rights reserved to Microsoft \n\n  \n\n\n\nZava Role Library \n\nLast Updated: 2023-03-05 \n\nChief Executive Officer \n\nJob Description: \n\nPosition: Chief Executive Officer \n\nCompany: Zava \n\n

In [19]:
import pandas as pd

activity_types = [{"type": a.type} for a in result.activity]

df = pd.DataFrame(activity_types)

print("Activity Log Steps")
df

Activity Log Steps


,type
0,modelQueryPlanning
1,searchIndex
2,searchIndex
3,searchIndex
4,agenticReasoning
5,modelAnswerSynthesis


In [20]:
activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("Activity Details")
print(activity_content)

Activity Details
[
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 2201,
    "input_tokens": 1455,
    "output_tokens": 64
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 885,
    "knowledge_source_name": "hrdocs-knowledge-source",
    "query_time": "2026-02-20T03:21:15.695Z",
    "count": 33,
    "search_index_arguments": {
      "search": "Responsibilities of the Zava CEO",
      "source_data_fields": [
        {
          "name": "snippet"
        },
        {
          "name": "uid"
        },
        {
          "name": "blob_path"
        }
      ],
      "search_fields": [],
      "semantic_configuration_name": "semantic-configuration"
    }
  },
  {
    "id": 2,
    "type": "searchIndex",
    "elapsed_ms": 311,
    "knowledge_source_name": "hrdocs-knowledge-source",
    "query_time": "2026-02-20T03:21:16.014Z",
    "count": 16,
    "search_index_arguments": {
      "search": "Role of the CEO at Zava",
      "source_data_fields": [
       

## Summary

You've completed Part 1! 🎉 You created a knowledge source, built a knowledge base with Azure OpenAI, and ran agentic retrieval to get citation-backed answers.

**Key concepts to remember:**
- Knowledge sources connect to your data (indexes, SharePoint, web, blob)
- Knowledge bases orchestrate AI models and data sources
- Agentic retrieval reasons about queries and synthesizes grounded answers
- Activity logs show how the agent processed your query

➡️ Continue to [Part 2: Multiple Knowledge Sources](part2-multiple-knowledge-sources.ipynb) to learn how to query multiple indexes and control source selection.